# Dijkstra's Algorithm Business Application Implementation
### Autumn Mizer, Jacob Minikel, John Poquette

This notebook contains a mock business application of finding nearest shelters given a users location. The user location is manually inputted as well as the shelters. In a real application, the users location would automatically be determined as well as active disaster shelters be pulled from the Red Cross API.

In [67]:
from collections import defaultdict
import heapq
import osmnx as ox
import json

ox.settings.use_cache = True # cache graphs
ox.settings.log_console = False

In [68]:
def dijkstra(graph, start, end=None):
    """
    Dijkstra's algorithm to find shortest paths.
    
    Parameters:
    - graph: defaultdict(list) where graph[u] = [(v, weight), ...]
    - start: Starting node ID
    - end: Optional end node ID (if None, compute distances to all nodes)
    
    Returns:
    - If end is specified: float (distance to end or inf if unreachable)
    - If end is None: dict mapping nodes to distances
    """
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    pq = [(0, start)]

    while pq:
        current_dist, u = heapq.heappop(pq)
        # Skip if we've found a better path already
        if current_dist > dist.get(u, float('inf')):
            continue

        # If end is specified and we've reached it, return the distance
        if end is not None and u == end:
            return current_dist

        # Process all neighbors of the current node
        for v, weight in graph.get(u, []):
            new_dist = current_dist + weight
            if new_dist < dist.get(v, float('inf')):
                dist[v] = new_dist
                heapq.heappush(pq, (new_dist, v))

    # If end was specified but not reached, return inf
    # If end was not specified, return distances to all nodes
    return float('inf') if end is not None else dist

The code segment below tracks previous nodes along the shortest path to reconstruct the full route from the target to the source.

In [69]:
def dijkstra_with_prev(graph, start, end):
    dist = {node: float('inf') for node in graph}
    prev = {node: None for node in graph}
    dist[start] = 0
    pq = [(0, start)]
    while pq:
        current_dist, u = heapq.heappop(pq)
        if u == end:
            return dist, prev
        for v, weight in graph[u]:
            new_dist = current_dist + weight
            if new_dist < dist[v]:
                dist[v] = new_dist
                prev[v] = u
                heapq.heappush(pq, (new_dist, v))
    return dist, prev

The code segment below converts an OSMnx MultiDiGraph to a dictionary. Each node maps to a list of (neighbor, edge_weight) tuples. THe length is in meters.

In [70]:
def osmnx_convert(G, weight='length'):
    adj = defaultdict(list)
    # graph is simplified and edges have 'length'
    for u, v, key, data in G.edges(keys=True, data=True):
        w = data.get(weight)
        if w is None:
            # if length missing, approximate with geometry
            if 'geometry' in data:
                w = data['geometry'].length
            else:
                w = 1.0
        adj[u].append((v, float(w)))
    # include nodes with no outgoing edges
    for n in G.nodes():
        if n not in adj:
            adj[n] = []
    return dict(adj)

A more reasistic application would be getting active shelters from the Red Cross API, although we need an API key for this that we need to apply for so it is not feasible for just a project.

https://www.redcross.org/get-help/disaster-relief-and-recovery-services/find-an-open-shelter.html

The 'data' below was taken from here. The addresses were put in a find my lat/lon. https://www.facebook.com/MKEOEM/posts/the-red-cross-now-has-two-shelters-open-in-milwaukee-for-those-who-have-been-aff/1158953626265370/

In [71]:
mock_shelters = [
    {"name": "Holler Park", "lat": 42.9510731, "lon": -87.9215229},
    {"name": "Washington Park Senior Center", "lat": 43.0496277, "lon": -87.9696743},
]

The code segment below downloads the road network for Milwaukee from OpenStreetMap (using osmnx)

In [72]:
place = "Milwaukee, Wisconsin, USA"
print("downloading Milwaukee road network...")
G = ox.graph_from_place(place, network_type="drive")
adj = osmnx_convert(G, weight="length")

print(f"graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")

downloading Milwaukee road network...
graph loaded: 13691 nodes, 37209 edges


The code segment below takes a manual location and maps it to the nearest node on the road graph. In a real application of this idea it would be automaticaly detected. Although there is nothing really stopping us from detecting location automatically.

In [73]:
lat, lon = 43.054413, -87.890029 # change source location manually
origin_node = ox.distance.nearest_nodes(G, lon, lat)
print("Nearest road node:", origin_node)

Nearest road node: 196677729


The code segment below maps the shelters to the nearest nodes on the road graph

In [74]:
print("mapping shelters to nearest road nodes...")
for sh in mock_shelters:
    sh["node"] = ox.distance.nearest_nodes(G, sh["lon"], sh["lat"])

mapping shelters to nearest road nodes...


The code segment below finds the nearest shelter from the users location. Each shelter distance is computed from the users location and the shelter with the best (shortest distance) is taken.

In [75]:
best = None
best_dist = float('inf')
for sh in mock_shelters:
    d = dijkstra(adj, origin_node, sh["node"])
    if d < best_dist:
        best_dist = d
        best = sh

if best is None:
    print("no shelters found")
    exit()

print("nearest shelter")
print("name:", best["name"])
print(f"distance {best_dist}m")
print("node:", best["node"])

nearest shelter
name: Washington Park Senior Center
distance 7303.451127726327m
node: 4030408235


The code segment below uses Dijkstra's algorithm with the previous nodes to generate a full route from the users location.

In [76]:
dist_map, prev_map = dijkstra_with_prev(adj, origin_node, best["node"])
path = []
cur = best["node"]
while cur is not None:
    path.append(cur)
    cur = prev_map[cur]
path.reverse()
print("path length:", len(path), "nodes.")

path length: 79 nodes.


The code segment below exports the route as a json file that can be used at geojson to actually visualize the route.

In [77]:
node_xy = {n: (data["x"], data["y"]) for n, data in G.nodes(data=True)}
coords = [(node_xy[n][0], node_xy[n][1]) for n in path]

geo = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "properties": {
            "origin_lat": lat,
            "origin_lon": lon,
            "shelter": best["name"],
            "distance_m": best_dist
        },
        "geometry": {
            "type": "LineString",
            "coordinates": coords
        }
    }]
}

with open("route.geojson", "w") as f:
    json.dump(geo, f, indent=2)

print("put saved geo in https://geojson.io")

put saved geo in https://geojson.io
